<a href="https://colab.research.google.com/github/1900690/yolo-traking/blob/main/yolo_traking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#果実トラッキング

※学習前で追跡できるのはリンゴ、オレンジなど代表的なもののみ

In [1]:
#@title 分析する動画をアップロードまたはサンプル動画(ユズ)で分析
from google.colab import files
import os
サンプル動画を使用する = False # @param {type:"boolean"}

if サンプル動画を使用する == True:
  if os.path.exists('/content/yuzu.mp4'):
     os.remove('/content/yuzu.mp4')
  !wget -q  https://github.com/1900690/yolo-traking/releases/download/sample/yuzu.mp4

else:
  uploaded_file = files.upload()
  uploaded_file_name = next(iter(uploaded_file))

Saving yuzu.mp4 to yuzu.mp4


In [2]:
#事前準備
!pip -q install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.5/870.5 kB 14.9 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO

# Configure the tracking parameters and run the tracker
model = YOLO("yolo11n.pt")
if サンプル動画を使用する == True:
  !yolo track model=yolo11n.pt source="/content/yuzu.mp4" conf=0.3, iou=0.5
else:
  file_name="/content/"+uploaded_file_name#★名前が挿入できないので工夫が必要
  !yolo track model=yolo11n.pt source=file_name conf=0.3, iou=0.5

WARNING ⚠️ argument 'conf=0.3,' does not require trailing comma ',', updating to 'conf=0.3'.
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 798, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 601, in track
    return self.predict(source=source, stream=stream, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 554, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/predictor.py", line 183, 